### Imports

In [12]:
import numpy as np

import sys

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib as mpl

from astropy import units as u
from astropy import constants as const
from astropy.table import QTable

from pathlib import Path
import os

sys.path.insert(0, '/home/emelie/Desktop/Codes/PETAR/PeTar-master/tools/')
from analysis import *

In [13]:
plt.rcParams.update({'xtick.labelsize':13, 'ytick.labelsize':13, 'axes.titlesize':16, 
                     'axes.grid':True, 'axes.labelsize':15, 'legend.fontsize':13})

# Functions

## Half-mass radius function

In [66]:
def halfmass_radius_cumsum(masses, radii):
    new_data_array = np.empty((len(masses), 2)) # Make an empty array
    new_data_array[:, 0] = masses # Fills the first column with masses
    new_data_array[:, 1] = radii # Fills the second columns with radii
    sorted_array = np.array(sorted(new_data_array, key=lambda x:x[1])) # Sort the data according to the radii
    
    cumsum_mass = np.cumsum(sorted_array[:, 0]) # Calculate a cumulative sum of all masses, shape(n_particles)
    
    hist_data = np.vstack((cumsum_mass, sorted_array[:, 1]))
    
    Mtot = np.sum(masses)
    half_mass = 0.5*Mtot
    
    difference = np.abs(cumsum_mass - half_mass)
    closest_mass = np.min(difference)
    
    position = np.where(difference==closest_mass)[0]
    
    r_halfmass = sorted_array[position, 1][0]
    
    return r_halfmass, hist_data

## Energy function

**NOTE!**
Copied from computational n-body simulation, it does however not account for the potential. Fix/check!!!

In [67]:
def energy(x, y, z, vx, vy, vz, masses):
    m = np.array(masses).flatten() * u.Msun # shape=(N, )
    
    v = np.sqrt(vx**2 + vy**2 + vz**2)
    
    # Fixing position vectors i and j, gives shape (N, 1, 3) and (1, N, 3)
    r = np.transpose(np.vstack((x, y, z)))    
    r_i = r[:, np.newaxis, :] * u.au
    r_j = r[np.newaxis, :, :] * u.au
    
    
    # Calculate distances
    d = r_i - r_j 
    
    #print(f'{d = }')
    
    
    absd = np.linalg.norm(d, axis=2)
    abs_d = absd[:, :, np.newaxis]
    
    #print(f'{abs_d = }')
    
    
    
    division = 1/(2*abs_d)
    
    # Replaces infs with 0
    division[np.isinf(division)] = 0
    

    P = ((-c.G*np.matmul(m, division)).flatten()).value # shape=(N, )
    
    #print(f'{P = }')
    
    for i in range(len(P)):
        if P[i] == -np.inf:
            P[i] = 0
    
    
    K = v**2/2 # shape=(N, )
    
    inside = P + K # shape=(N, )
    #print(f'{inside = }')
    
    E_tot = np.matmul(m, inside).value
    

    
    return E_tot
    
    
def energy_conservation(E, E_in):
    return np.abs(E - E_in)/np.abs(E_in)

## The function

**Things I want to put into the function**

- Number of timesteps
- Times for output
- Name of run
- Folder for the data files
- Check that the number of particles is conserved!


**The output of the function**

- mean $x$, $y$, $z$
- mean $r = \sqrt{x² + y² + z²}$
- mean $v_x$, $v_y$, $v_z$
- halfmass radius
- momentum over time?
- energy over time?
- headers
- \# of part conserved

In [111]:
def extract_data(name_run, name_folder, tmax, toutput):
    """
    Extracts data from petar snapshot files and checks the data
    ------------------------------------------------------------
    Parameters:
    -----------
    name_run: str
        Name of the run. NOTE! Do not forget to add an r before the string!!!
        
    name_folder: str
        Name of the folder in which the data is kept. NOTE! Do not forget to add an r before the string!!!
        
    tmax: int
        Number of timesteps
    
    toutput: int
        Which timesteps to give snapshots
        
        
    Output:
    --------
    all_data: array
        An array with all data for all particles and all snapshots. The order of the columns is 
        [mass, x, y, z, r, vx, vy, vz, v, r_search, mass_bk, status, acc_sofs x, acc_soft y, acc_soft z, 
        pot_tot, pot_soft, pot_ext].
        
    header_values: array
        An array with all data from the headers of the snapshots. The order of the columns is 
        [number of particles, time, x_offset, y_offset, z_offset, vx_offset, vy_offset, vz_offset]
    
    extra_data: array
        Calculated extra data that is relevant for the analysis:
        [half_mass_radius, r_mean_particles, r_mean_cluster, v_mean]
        
    hist_data: array
        Data for making a cumulative mass vs radius plot 
        
    """
    n_files = int((tmax/toutput) + 1)
    
    
    # Fixing basic path
    notebook_path = Path.cwd()

    data_path = Path('Result_files')

    total_path = os.path.join(notebook_path, data_path)
    
    # Importing header
    header_values = np.empty((8, n_files)) # (quantities, timesteps)
    extra_data = np.zeros((4, n_files)) # (quantities, timesteps)

    for i in range(n_files):
        header = PeTarDataHeader(total_path+f'/{name_folder}/{name_run}.{i}', 
                                       external_mode='galpy', snapshot_format='ascii')
        header_values[0, i] = header.n # Number of particles
        header_values[1, i] = header.time # Time
        
        x_mean = header.pos_offset[0] # Cluster mean x position
        y_mean = header.pos_offset[1] # Cluster mean y position
        z_mean = header.pos_offset[2] # Cluster mean z position
        header_values[2, i] = x_mean 
        header_values[3, i] = y_mean
        header_values[4, i] = z_mean
        
        r_mean_cluster = np.sqrt(x_mean**2 + y_mean**2 + z_mean**2)
        
        extra_data[2, i]
        
        
        vx_mean = header.vel_offset[0] # Cluster mean velocity in x position
        vy_mean = header.vel_offset[1] # Cluster mean velocity in y position
        vz_mean = header.vel_offset[2] # Cluster mean velocity in z position
        header_values[5, i] = vx_mean 
        header_values[6, i] = vy_mean 
        header_values[7, i] = vz_mean
        
        v_mean_cluster = np.sqrt(vx_mean**2 + vy_mean**2 + vz_mean**2)
        extra_data[3, i] = v_mean_cluster
        
        
        
        
    # Importing data into 3D array
    all_data = np.empty((int(header_values[0, 0]), 21, n_files)) # (particles, quantities, timesteps)
    n_particles = np.empty((n_files))
    hist_data = np.zeros((2, int(header_values[0, 0]), n_files))
    
    for i in range(n_files):
        # Importing the data
        particles = Particle(external_mode='galpy', interupt_mode='bse')
        particles.loadtxt(total_path+f'/{name_folder}/{name_run}.{i}', skiprows=1)
        all_data[:, 0, i] = particles.mass # mass
        
        # Working with the positions
        x = particles.pos[:, 0] 
        y = particles.pos[:, 1] 
        z = particles.pos[:, 2] 
        
        # Adjusting the positions of the particles to the position of the cluster
        all_data[:, 1, i] = x + header_values[2, i] # Galactic centre in origin frame
        all_data[:, 2, i] = y + header_values[3, i]
        all_data[:, 3, i] = z + header_values[4, i]
        
        r = np.sqrt(x**2 + y**2 + z**2) # Cluster centered around the origin frame
        all_data[:, 4, i] = r
        
        extra_data[1, i] = np.mean(r, axis=0) # r_mean_particles
    
        #extra_data[1, i] = np.mean(x, axis=0) # x_mean
        #extra_data[2, i] = np.mean(y, axis=0) # y_mean
        #extra_data[3, i] = np.mean(z, axis=0) # z_mean
        
        # Working with the velocities
        vx = particles.vel[:, 0] # vx
        vy = particles.vel[:, 1] # vy
        vz = particles.vel[:, 2] # vz
        all_data[:, 5, i] = vx
        all_data[:, 6, i] = vy
        all_data[:, 7, i] = vz
        
        #extra_data[5, i] = np.mean(vx, axis=0) # vx_mean
        #extra_data[6, i] = np.mean(vy, axis=0) # vy_mean
        #extra_data[7, i] = np.mean(vz, axis=0) # vz_mean
    
        v = np.sqrt(vx**2 + vy**2 + vz**2)
        all_data[:, 8, i] = v
        
        # The rest of the data
        all_data[:, 9, i] = particles.r_search # neighbour searching radius
        all_data[:, 10, i] = particles.id # particle id
        all_data[:, 11, i] = particles.mass_bk # Artificial particle parameter
        all_data[:, 12, i] = particles.status # Artificial particle parameter
        all_data[:, 13, i] = particles.r_in # inner changeover radius
        all_data[:, 14, i] = particles.r_out # outer changeover radius
        all_data[:, 15, i] = particles.acc_soft[:, 0] # long-range acceleration in x
        all_data[:, 16, i] = particles.acc_soft[:, 1] # long-range acceleration in y
        all_data[:, 17, i] = particles.acc_soft[:, 2] # long-range acceleration in z
        all_data[:, 18, i] = particles.pot # total potential
        all_data[:, 19, i] = particles.pot_soft # long-range(?) potential
        all_data[:, 20, i] = particles.pot_ext # external potential
        
        # Checking number of particles
        n_particles[i] = len(particles.mass)
        
        # Half-mass radius
        extra_data[0, i], hist_data[:, :, i] = halfmass_radius_cumsum(particles.mass, r)
    
    # Checking if the number of particles is the same throughout the simulation
    n_part_same = np.all(n_particles == n_particles[0])
    print(f'Number of particles is conserved: {n_part_same}')
    
    # Calculating useful quantities    
    
    
    
    # Conservation of momentum
    
    
    
    # Conservation of energy
    # E(t)-E(0)/E(0)?
    
    
    
    return all_data, header_values, extra_data, hist_data
        
        

## Calculating means

In [ ]:
def means_calc(header_array):
    v_mean = np.sqrt(header_array[-3, :]**2 +header_array[-2, :]**2 +header_array[-1, :]**2)
    
    r_mean = np.sqrt(header_array[2, :]**2 + header_array[3, :]**2 + header_array[4, :]**2)
    
    return v_mean, r_mean

## Plotting Means

In [5]:
def means_plots(which, header_array, mean, t_max, run, fig_width=14, fig_height=10, save=True):
    """
    Parameters:
    -----------

    which: str
            If the plots should contain velocities or positions
            
    header_array: array
            Array with header values: id, N, t, x, y, z, vx, vy, vz
            
    mean: float
            Mean value
            
    t_max: int or float
            Maximum time of simulation
            
    run: str
            Name of run
            
    fig_width: int or float
            Width of figure
            
    fig_height: int or float
            Height of figure
            
    save: str
            If figure should be saved
    
    """
    text = t_max/10
    tmin = 0-text
    tmax = t_max+text
    
    
    if which=='vel':
        fig, ax = plt.subplots(2, 2, figsize=(fig_width, fig_height))

        ax[0,0].plot(header_array[1, :], mean, color='b', marker='o')

        ax[0,0].set_xlabel('Time [Myr]')
        ax[0,0].set_ylabel(r'$v_{mean}$ [km/s]')
        ax[0,0].set_title('Mean velocity per timestep')
        ax[0,0].set_xlim(tmin, tmax)



        ax[0,1].plot(header_array[1, :], header_array[-3, :], color='b', marker='o')

        ax[0,1].set_xlabel('Time [Myr]')
        ax[0,1].set_ylabel(r'$v_{mean, x}$ [km/s]')
        ax[0,1].set_title('Mean velocity in x per timestep')
        ax[0,1].axhline(0, color='k', zorder=0)
        ax[0,1].set_xlim(tmin, tmax)


        ax[1,0].plot(header_array[1, :], header_array[-2, :], color='b', marker='o')

        ax[1,0].set_xlabel('Time [Myr]')
        ax[1,0].set_ylabel(r'$v_{mean, y}$ [km/s]')
        ax[1,0].set_title('Mean velocity in y per timestep')
        ax[1,0].axhline(0, color='k', zorder=0)
        ax[1,0].set_xlim(tmin, tmax)



        ax[1,1].plot(header_array[1, :], header_array[-1, :], color='b', marker='o')

        ax[1,1].set_xlabel('Time [Myr]')
        ax[1,1].set_ylabel(r'$v_{mean, z}$ [km/s]')
        ax[1,1].set_title('Mean velocity in z per timestep')
        ax[1,1].axhline(0, color='k', zorder=0)
        ax[1,1].set_xlim(tmin, tmax)

        plt.tight_layout()
        
        if save:
            plt.savefig(f'Velocity_plots_{run}.png', bbox_inches='tight')
            
        plt.show()
        
        
        
        
    if which=='pos':
        fig, ax = plt.subplots(2, 2, figsize=(fig_width, fig_height))

        ax[0,0].plot(header_array[1, :], mean, color='b', marker='o')

        ax[0,0].set_xlabel('Time [Myr]')
        ax[0,0].set_ylabel(r'$r_{mean}$ [pc]')
        ax[0,0].set_title('Mean r per timestep')
        ax[0,0].set_xlim(tmin, tmax)
        #ax[0,0].set_ylim(ymin=-1000, ymax=23000)



        ax[0,1].plot(header_array[1, :], header_array[2, :], color='b', marker='o')

        ax[0,1].set_xlabel('Time [Myr]')
        ax[0,1].set_ylabel(r'$x_{mean}$ [pc]')
        ax[0,1].set_title('Mean x per timestep')
        ax[0,1].axhline(0, color='k', zorder=0)
        ax[0,1].set_xlim(tmin, tmax)
        #ax[0,1].set_ylim(ymin=-21000, ymax=21000)


        ax[1,0].plot(header_array[1, :], header_array[3, :], color='b', marker='o')

        ax[1,0].set_xlabel('Time [Myr]')
        ax[1,0].set_ylabel(r'$y_{mean}$ [pc]')
        ax[1,0].set_title('Mean y per timestep')
        ax[1,0].axhline(0, color='k', zorder=0)
        ax[1,0].set_xlim(tmin, tmax)
        #ax[1,0].set_ylim(ymin=-21000, ymax=21000)



        ax[1,1].plot(header_array[1, :], header_array[4, :], color='b', marker='o')

        ax[1,1].set_xlabel('Time [Myr]')
        ax[1,1].set_ylabel(r'$z_{mean}$ [pc]')
        ax[1,1].set_title('Mean z per timestep')
        ax[1,1].axhline(0, color='k', zorder=0)
        ax[1,1].set_xlim(tmin, tmax)
        #ax[1,1].set_ylim(ymin=-21000, ymax=21000)

        plt.tight_layout()
        
        if save:
            plt.savefig(f'Position_plots_{run}.png', bbox_inches='tight')
        
        plt.show()
        

# Tests

In [108]:
#run_at_centre_data, run_at_centre_header, centre_extra_data, centre_hist_data = extract_data(r'mcluster_at_centre_data', r'Trialrun_1000part_at_centre', 30, 1)
#print(run_at_centre_data.shape)
#print(centre_hist_data.shape)
#print(centre_extra_data.shape)

#centre_halfmass_late = centre_extra_data[0, 6]
#centre_halfmass_early = centre_extra_data[0, 1]
#print(centre_halfmass)
#print(centre_hist_data[0, -1, 6])
#print(np.sum(run_at_centre_data[:, 0, 6]))
#print()


#run_above_disc_data, run_above_disc_header, above_extra_data, above_hist_data = extract_data(r'mcluster_above_disc_data', r'Trialrun_1000part_above_disc', 6, 1)
#print(run_above_disc_data.shape)
#print(above_hist_data.shape)
#print(above_extra_data.shape)

#above_halfmass_late = above_extra_data[0, 6]
#above_halfmass_early = above_extra_data[0, 1]
#print(above_halfmass)

In [109]:
#test_fig1, testax1 = plt.subplots(1, 2, figsize=(14, 5))


#testax1[0].minorticks_on()
#testax1[0].plot(centre_hist_data[1, :, 1], centre_hist_data[0, :, 1], color='b')
#testax1.scatter(sorted_data[position[0], 7], cumsum_mass[position[0]], color='r')

#testax1[0].plot(above_hist_data[1, :, 1], above_hist_data[0, :, 1], color='r')
#testax1.scatter(sorted_data2[position2[0], 7], cumsum_mass2[position2[0]], color='g')
#testax1[0].set_xlabel('r')
#testax1[0].set_ylabel('Sum of M')
#testax1[0].set_title('Cumsum hist early')
#testax1[0].axvline(centre_halfmass_early, color='aqua', linestyle='dashed')
#testax1[0].axvline(above_halfmass_early, color='tomato', linestyle='dashed')
#testax1[0].grid(which='both')


#testax1[1].minorticks_on()
#testax1[1].plot(centre_hist_data[1, :, 6], centre_hist_data[0, :, 6], color='b')
#testax1.scatter(sorted_data[position[0], 7], cumsum_mass[position[0]], color='r')

#testax1[1].plot(above_hist_data[1, :, 6], above_hist_data[0, :, 6], color='r')
#testax1.scatter(sorted_data2[position2[0], 7], cumsum_mass2[position2[0]], color='g')
#testax1[1].set_xlabel('r')
#testax1[1].set_ylabel('Sum of M')
#testax1[1].set_title('Cumsum hist late')
#testax1[1].axvline(centre_halfmass_late, color='aqua', linestyle='dashed')
#testax1[1].axvline(above_halfmass_late, color='tomato', linestyle='dashed')
#testax1[1].grid(which='both')

#plt.show()

In [107]:
#my_data, my_header = extract_data(r'mcluster_at_centre_data', r'Trialrun_1000part_at_centre', 30, 1)

#print(np.shape(my_data))
#print(halfmass_radius_cumsum(my_data[:, 0, 0], my_data[:, 7, 0]))
#print(my_extra_data[0, :])
#print(my_extra_data.shape)#

In [105]:
#data_t1 = my_data[:, :, 6]
#print(np.shape(data_t1))
#print(type(data_t1))
#print(data_t1[:, 7])
#print(data_t1[:, 0])
#print()
#print(min(data_t1[:, 7]))
#print(max(data_t1[:, 7]))
#print()


#my_r = my_data[:, 7, 0]

#sorted_data = np.array(sorted(data_t1, key=lambda x:x[7]))
#print(np.shape(sorted_data))
#print(type(sorted_data))
#print(sorted_data[:,7])
#print(sorted_data[:,0])

In [106]:
#cumsum_mass = np.cumsum(sorted_data[:, 0])
#print(cumsum_mass[3200:3300])
#print(len(cumsum_mass))
#print(np.sum(data_t1[:, 0]))
#print(cumsum_mass[-1])
#Mtot = cumsum_mass[-1]
#half_mass = 0.5*Mtot
#print(f'{half_mass = }')

#half_mass_diff = np.abs(cumsum_mass - half_mass)
#closest = np.min(half_mass_diff)
#print(closest)
#position = np.where(half_mass_diff==closest)[0]
#print(half_mass_diff[position])
#print(cumsum_mass[position])
#print(sorted_data[position, 7], position)

In [11]:
#r_halfmass = halfmass_radius_finder(my_data[:, 0, 0], my_data[:, 7, 0])
#print()
#print(r_halfmass)
#print()
#r_halfmass2 = halfmass_radius(my_data[:, 0, 5], my_radii)
#print()
#print(r_halfmass2)

r_test = 6.17909780681924
difference = 808.6573131521109
r_test = 9.268646710228861
difference = 110.85027228494118
r_test = 9.268646710228861
difference = 110.85027228494118

9.268646710228861


# Unused functions

In [ ]:
def halfmass_radius_finder(masses, radii):
    Mtot = np.sum(masses)
    
    r_max = np.max(radii)
    r_test = 0.5*r_max
    r_tests = []
    r_tests.append(r_test)
    n_part_inside = []
    
    masses_inside = masses[radii<=r_test]
    Mtot_inside = np.sum(masses_inside)
    n_part_inside.append(len(masses_inside))
    #print(len(masses_inside))
    difference = 0.5*Mtot-Mtot_inside
    
    if np.abs(difference)<=1e-1:
        return r_test
    
    else:
        if difference<0:
            r_test = r_test - 0.5*r_test
            r_tests.append(r_test)
            
        elif difference>0:
            r_test = r_test + 0.5*r_test
            r_tests.append(r_test)
            
        
        masses_inside = masses[radii<=r_test]
        n_part_inside.append(len(masses_inside))
        #print(len(masses_inside))
        Mtot_inside = np.sum(masses_inside)
        difference = 0.5*Mtot-Mtot_inside
        it = 1
        while np.abs(difference)>1e-1:
            print(f'{r_test = }')
            print(f'{difference = }')
            if difference<0:
                r_test = r_test - 0.5*np.abs(r_test-r_tests[it-1])
                
                r_tests[it] = r_test
                r_tests.append(r_test)
                
                masses_inside = masses[radii<=r_test]
                #print(len(masses_inside))
                n_part_inside.append(len(masses_inside))
                if len(masses_inside)==n_part_inside[it-1]:
                    return r_test
                    break
                Mtot_inside = np.sum(masses_inside)
                difference = 0.5*Mtot-Mtot_inside
                it = it+1
                
                
            elif difference>0:
                r_test = r_test + 0.5*np.abs(r_test-r_tests[it-1])
                
                r_tests[it] = r_test
                r_tests.append(r_test)
                masses_inside = masses[radii<=r_test]
                #print(len(masses_inside))
                n_part_inside.append(len(masses_inside))
                if len(masses_inside)==n_part_inside[it-1]:
                    return r_test
                    break
                Mtot_inside = np.sum(masses_inside)
                difference = 0.5*Mtot-Mtot_inside
                it = it+1
                
        return r_test